In [1]:
from openpyxl import load_workbook
import os
from xls2xlsx import XLS2XLSX
import requests
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm


In [ ]:
data_dict = {
    'd2010.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2010/publishedweek2010.xls',
    'd2011.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2011/publishedweek2011.xls',
    'd2012.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2012/publishedweek2012.xls',
    'd2013.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2013/publishedweek2013.xls',
    'd2014.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2014/publishedweek2014.xls',
    'd2015.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2015/publishedweek2015.xls',
    'd2016.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2016/publishedweek522016.xls',
    'd2017.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2017/publishedweek522017.xls',
    'd2018.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2018/publishedweek522018withupdatedrespiratoryrow.xls',
    'd2019.xls':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2019/publishedweek522019.xls',
    'd2020.xlsx':'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fbirthsdeathsandmarriages%2fdeaths%2fdatasets%2fweeklyprovisionalfiguresondeathsregisteredinenglandandwales%2f2020/publishedweek532020.xlsx',
}

for fname in data_dict:
    r = requests.get(data_dict[fname], allow_redirects=True)
    open(fname, 'wb').write(r.content)
        
    if('xlsx' not in fname):
        print('Converting %s to xlsx format'%fname)
        x2x = XLS2XLSX(fname)
        x2x.to_xlsx(fname.split('.')[0] + '.xlsx')
        os.remove(fname)
# end for    

Converting d2010.xls to xlsx format
Converting d2011.xls to xlsx format
Converting d2012.xls to xlsx format
Converting d2013.xls to xlsx format
Converting d2014.xls to xlsx format


In [ ]:
class YearlyData:
    def __init__(self, 
                 work_sheet=None, week_number_cells='',
                 week_ended_cells='', total_deaths_all_ages='',
                 age_groups_cells='', deaths_by_age_group_cells=''):

        self.week_number = None
        self.week_ended = None
        self.total_deaths_all_ages = None
        self.age_groups = None
        self.deaths_by_age_group = None
        
        ranges_dict = {'week_number': week_number_cells, 
                       'week_ended': week_ended_cells, 
                       'total_deaths_all_ages': total_deaths_all_ages, 
                       'age_groups': age_groups_cells, 
                       'deaths_by_age_group': deaths_by_age_group_cells}
        
        for item in ranges_dict:
            self.__dict__[item] = []
            range_tokens = ranges_dict[item].split(':')
            for row in work_sheet[slice(range_tokens[0], range_tokens[1])]:
                cols = []
                for cell in row:
                    cols.append(cell.value)        
                # end for
                self.__dict__[item].append(cols)        
            # end for
            self.__dict__[item] = np.squeeze(np.array(self.__dict__[item]))
            #print (np.shape(self.__dict__[item]))
            #print(self.__dict__[item])
        # end for     
        
        #sanity checks
        assert(np.shape(self.week_number) == np.shape(self.week_ended))
        assert(np.shape(self.deaths_by_age_group) == 
               (np.shape(self.age_groups)[0], np.shape(self.week_number)[0]))
    # end func
# end class


In [ ]:
data_dict = {}

d2020 = load_workbook(filename='d2020.xlsx', read_only=True)
data_dict[2020] = YearlyData(d2020['Weekly figures 2020'], 
                   week_number_cells='C5:BC5',
                   week_ended_cells='C6:BC6',
                   total_deaths_all_ages='C9:BC9',
                   age_groups_cells='B22:B41',
                   deaths_by_age_group_cells='C22:BC41')

d2019 = load_workbook(filename='d2019.xlsx', read_only=True)
data_dict[2019] = YearlyData(d2019['Weekly figures 2019'], 
                   week_number_cells='C4:BB4',
                   week_ended_cells='C5:BB5',
                   total_deaths_all_ages='C8:BB8',
                   age_groups_cells='B16:B22',
                   deaths_by_age_group_cells='C16:BB22')

d2018 = load_workbook(filename='d2018.xlsx', read_only=True)
data_dict[2018] = YearlyData(d2018['Weekly figures 2018'], 
                   week_number_cells='C4:BB4',
                   week_ended_cells='C5:BB5',
                   total_deaths_all_ages='C8:BB8',
                   age_groups_cells='B16:B22',
                   deaths_by_age_group_cells='C16:BB22')

d2017 = load_workbook(filename='d2017.xlsx', read_only=True)
data_dict[2017] = YearlyData(d2017['Weekly figures 2017'], 
                   week_number_cells='C4:BB4',
                   week_ended_cells='C5:BB5',
                   total_deaths_all_ages='C8:BB8',
                   age_groups_cells='B16:B22',
                   deaths_by_age_group_cells='C16:BB22')

d2016 = load_workbook(filename='d2016.xlsx', read_only=True)
data_dict[2016] = YearlyData(d2016['Weekly figures 2016'], 
                   week_number_cells='C4:BB4',
                   week_ended_cells='C5:BB5',
                   total_deaths_all_ages='C8:BB8',
                   age_groups_cells='B16:B22',
                   deaths_by_age_group_cells='C16:BB22')

d2015 = load_workbook(filename='d2015.xlsx', read_only=True)
data_dict[2015] = YearlyData(d2015['Weekly Figures 2015'], 
                   week_number_cells='B4:BB4',
                   week_ended_cells='B5:BB5',
                   total_deaths_all_ages='B8:BB8',
                   age_groups_cells='A16:A22',
                   deaths_by_age_group_cells='B16:BB22')

d2014 = load_workbook(filename='d2014.xlsx', read_only=True)
data_dict[2014] = YearlyData(d2014['Weekly Figures 2014'], 
                   week_number_cells='B3:BA3',
                   week_ended_cells='B4:BA4',
                   total_deaths_all_ages='B7:BA7',
                   age_groups_cells='A16:A22',
                   deaths_by_age_group_cells='B16:BA22')

d2013 = load_workbook(filename='d2013.xlsx', read_only=True)
data_dict[2013] = YearlyData(d2013['Weekly Figures 2013'], 
                   week_number_cells='B4:BA4',
                   week_ended_cells='B5:BA5',
                   total_deaths_all_ages='B8:BA8',
                   age_groups_cells='A16:A22',
                   deaths_by_age_group_cells='B16:BA22')

d2012 = load_workbook(filename='d2012.xlsx', read_only=True)
data_dict[2012] = YearlyData(d2012['Weekly Figures 2012'], 
                   week_number_cells='B4:BA4',
                   week_ended_cells='B5:BA5',
                   total_deaths_all_ages='B8:BA8',
                   age_groups_cells='A16:A22',
                   deaths_by_age_group_cells='B16:BA22')

d2011 = load_workbook(filename='d2011.xlsx', read_only=True)
data_dict[2011] = YearlyData(d2011['Weekly Figures 2011'], 
                   week_number_cells='B4:BA4',
                   week_ended_cells='B5:BA5',
                   total_deaths_all_ages='B8:BA8',
                   age_groups_cells='A17:A23',
                   deaths_by_age_group_cells='B17:BA23')

d2010 = load_workbook(filename='d2010.xlsx', read_only=True)
data_dict[2010] = YearlyData(d2010['Weekly Figures 2010'], 
                   week_number_cells='B4:BA4',
                   week_ended_cells='B5:BA5',
                   total_deaths_all_ages='B8:BA8',
                   age_groups_cells='A16:A22',
                   deaths_by_age_group_cells='B16:BA22')


In [ ]:
# Font settings
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='x-large')
plt.rc('ytick', labelsize='x-large')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches((10,5))
yearly_deaths = []
for item in sorted(data_dict.keys()):
    yearly_deaths.append([item, np.sum(data_dict[item].total_deaths_all_ages)])
# end for
yearly_deaths = np.array(yearly_deaths)
ax.plot(yearly_deaths[:,0], yearly_deaths[:,1])
ax.set_ylim([0,7e5])

for i in range(len(yearly_deaths)):
    ax.scatter(yearly_deaths[i, 0], yearly_deaths[i, 1], marker='+', c='grey', 
                label='%d : %d'%(yearly_deaths[i,0], yearly_deaths[i,1]))
# end for

ax.set_xlabel('Year', fontsize=20)
ax.set_ylabel('Total deaths', fontsize=20)
ax.legend(loc='best', bbox_to_anchor = (1.1, 0.9), fontsize=12)

In [ ]:
colors = cm.rainbow(np.linspace(0,1,len(data_dict)))
fig, ax = plt.subplots()
fig.set_size_inches((10,5))

for item,color in zip(sorted(data_dict.keys()), colors):
    ax.plot(data_dict[item].week_number, 
            data_dict[item].total_deaths_all_ages, c=color, 
            label='%d   (%d)'%(item, np.sum(data_dict[item].total_deaths_all_ages)))
    print(np.sum(data_dict[item].total_deaths_all_ages))
# end for

ax.fill_betweenx([0,25000], 12,26, color='r', alpha=0.2, label='First lockdown')
ax.fill_betweenx([0,25000], 44,48, color='darkred', alpha=0.2, label='Second lockdown')

ax.set_xlabel('Week number', fontsize=20)
ax.set_ylabel('Weekly deaths', fontsize=20)


ax.legend(loc='best', bbox_to_anchor = (1.1, 1), 
          fontsize=12, title='Year (total deaths)', 
          title_fontsize=15)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches((10,5))

y2020_alt = np.zeros((7,data_dict[2020].deaths_by_age_group.shape[1]))


y2020_alt[0,:] = data_dict[2020].deaths_by_age_group[0,:]
y2020_alt[1,:] = np.sum(data_dict[2020].deaths_by_age_group[1:4, :], axis=0)
y2020_alt[2,:] = np.sum(data_dict[2020].deaths_by_age_group[4:10, :], axis=0)
y2020_alt[3,:] = np.sum(data_dict[2020].deaths_by_age_group[10:14, :], axis=0)
y2020_alt[4,:] = np.sum(data_dict[2020].deaths_by_age_group[14:16, :], axis=0)
y2020_alt[5,:] = np.sum(data_dict[2020].deaths_by_age_group[16:18, :], axis=0)
y2020_alt[6,:] = np.sum(data_dict[2020].deaths_by_age_group[18:, :], axis=0)

y2020_alt_mean = np.mean(y2020_alt, axis=1)
y2020_alt_std = np.std(y2020_alt, axis=1)
ax.plot(np.arange(len(y2020_alt_mean)), y2020_alt_mean, color='r', label='2020')
ax.fill_between(np.arange(len(y2020_alt_mean)), y2020_alt_mean-y2020_alt_std, 
                 y2020_alt_mean+y2020_alt_std, 
                 color='r', alpha=0.1, label='$\pm 1\sigma$ envelope (2020)')

y2018_mean = np.mean(data_dict[2018].deaths_by_age_group, axis=1)
y2018_std = np.std(data_dict[2018].deaths_by_age_group, axis=1)
ax.plot(np.arange(len(y2018_mean)), y2018_mean, color='b', label='2018')
ax.fill_between(np.arange(len(y2018_mean)), y2018_mean-y2018_std, y2018_mean+y2018_std, 
                 color='b', alpha=0.1, label='$\pm 1\sigma$ envelope (2018)')

labels = data_dict[2018].age_groups
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)

ax.set_xlabel('Age bracket [years]', fontsize=20)
ax.set_ylabel('Mean weekly deaths', fontsize=20)

ax.legend(fontsize=12)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches((10,5))

colors = cm.rainbow(np.linspace(0,1,len(data_dict[2020].deaths_by_age_group)))
for i,color in zip(range(len(data_dict[2020].age_groups)), colors):
    ax.plot(np.arange(data_dict[2020].deaths_by_age_group.shape[1]), 
            data_dict[2020].deaths_by_age_group[i,:], color=color,
            label=data_dict[2020].age_groups[i])
# end for

ax.fill_betweenx([0,6000], 12,26, color='r', alpha=0.2, label='First lockdown')
ax.fill_betweenx([0,6000], 44,48, color='darkred', alpha=0.2, label='Second lockdown')

ax.set_xlabel('Week number (2020)', fontsize=20)
ax.set_ylabel('Weekly deaths', fontsize=20)
ax.legend(fontsize=12, bbox_to_anchor = (1.0, 1.0), ncol=2, 
          title='Age bracket [years]', title_fontsize=15)

print(np.sum(data_dict[2020].deaths_by_age_group[-2:,12:26]) - 
      np.sum(data_dict[2018].deaths_by_age_group[-1:,12:26]))    

In [ ]:
a = np.mean([np.sum(data_dict[2019].deaths_by_age_group[-2:,:]),
        np.sum(data_dict[2018].deaths_by_age_group[-2:,:]),
        np.sum(data_dict[2017].deaths_by_age_group[-2:,:]),
        np.sum(data_dict[2016].deaths_by_age_group[-2:,:]),
        np.sum(data_dict[2015].deaths_by_age_group[-2:,:])])

print(a)

b = np.sum(data_dict[2020].deaths_by_age_group[-4:, :])

print(b)